In [1]:
# 새로 수정된 버전입니다.

In [17]:
# 필요한 모듈을 import 한다. 
import numpy as np
from stl import mesh
import trimesh
import os
from scipy import ndimage

In [18]:
# 한번 확인하는 코드입니다.
# 기존 데이터를 불러와서 shape와 0과 1의 갯수 확인 / 이를 보간법을통해 다시 shape를 알맞게 조절 (이때 bilinear interpolation을 써서 문제가 되었음).
# order=0 으로 하여 nearest-neighbor interpolation 방법으로 보간법을 사용해 1과 0의 비율을 맞춰줌 


import numpy as np
import trimesh
from scipy import ndimage

# STL 파일을 읽습니다
mesh = trimesh.load_mesh("stl_data/C57289014.stl")

# STL 메쉬를 voxel grid로 변환합니다.
voxels = mesh.voxelized(pitch=0.05)

# voxel grid를 numpy array로 변환합니다.
array = voxels.matrix

print(f'Original shape: {array.shape}')  # 변환 전 shape 출력
# 1의 개수를 세어서 출력합니다
num_ones = np.count_nonzero(array)
print(f'Number of 1s: {num_ones}')

# 0의 개수를 세어서 출력합니다
num_zeros = array.size - num_ones
print(f'Number of 0s: {num_zeros}')

# 각 차원에 대한 증가 비율을 계산합니다.
zoom_ratio = np.array([64, 64, 64]) / array.shape
# 선형 보간을 사용하여 배열 크기를 변경합니다.
resized_array = ndimage.zoom(array, zoom_ratio, order=0) # k mean clustering


print(f'Resized shape: {resized_array.shape}') 
num_ones = np.count_nonzero(resized_array)
print(f'Number of 1s: {num_ones}')
num_zeros = resized_array.size - num_ones
print(f'Number of 0s: {num_zeros}')


Original shape: (25, 32, 32)
Number of 1s: 6830
Number of 0s: 18770
Resized shape: (64, 64, 64)
Number of 1s: 65896
Number of 0s: 196248


In [19]:
stl_dir = 'stl_data'
npy_dir = 'npy_data'

# npy_data 폴더가 생성
if not os.path.exists(npy_dir):
    os.makedirs(npy_dir)


for filename in os.listdir(stl_dir):
    if filename.endswith('.stl'):

        stl_path = os.path.join(stl_dir, filename)
        mesh = trimesh.load_mesh(stl_path)

        # (너무 오래걸릴시, 0.5정도로 먼저 저장해보고, 잘작동되면 0.025 또는 0.05정도로 파일저장)
        voxels = mesh.voxelized(pitch=0.05)
        
        # 결과 배열을 int 타입으로 변환합니다.
        array = voxels.matrix.astype(int)
        
        # 각 차원에 대한 증가 비율을 계산합니다.
        zoom_ratio = np.array([64, 64, 64]) / array.shape
        # 선형 보간을 사용하여 배열 크기를 변경합니다.
        resized_array = ndimage.zoom(array, zoom_ratio, order=0) # k mean clustering
        
        # npy 파일로 저장합니다.
        npy_path = os.path.join(npy_dir, filename.replace('.stl', '.npy'))
        np.save(npy_path, resized_array)

print("모두 저장완료.")

모두 저장완료.


In [22]:
npy_dir = 'npy_data'

npy_count = 0

for filename in os.listdir(npy_dir):
    if filename.endswith('.npy'):
    
        npy_count += 1
        
        # npy 파일 로드
        npy_path = os.path.join(npy_dir, filename)
        data = np.load(npy_path)
        
        # 파일이 너무 많은경우 아래 코드 주석처리 (저장된 npy 모양 출력확인용)
        print(f'Shape of {filename}:', data.shape)

# 총 npy 파일의 개수를 출력합니다.
print('\nTotal number of npy files:', npy_count)

Shape of C57289014.npy: (64, 64, 64)
Shape of LVZB011.npy: (64, 64, 64)
Shape of B14120054.npy: (64, 64, 64)
Shape of BD5312113A.npy: (64, 64, 64)

Total number of npy files: 4
